# **Preparação do ambiente Colab**

In [ ]:
# Importando as bibliotecas
import ee
from google.colab import drive
import geopandas as gpd
import geemap

In [ ]:
# Autenticação do Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-lucasviturisantarosa')

In [ ]:
# Montar Google Drive
drive.mount('/content/drive')

In [ ]:
# Exemplo de download para os limites de um país
aoi = ee.FeatureCollection('FAO/GAUL/2015/level0') \
         .filter(ee.Filter.eq('ADM0_NAME', 'Brazil')) \
         .geometry()

aoi

# **GroundWater Storage**

https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V022_CLSM_G025_DA1D#description

In [ ]:
# Definir os anos de interesse
anos = range(2003, 2023)  # Exemplo de 2018 a 2023

# Obter a geometria como lista de coordenadas para a exportação
region = aoi

# Definir o dataset e aplicar filtros de data
dataset = ee.ImageCollection('NASA/GLDAS/V022/CLSM/G025/DA1D') \
            .filter(ee.Filter.date('2003-01-01', '2023-12-31')) \
            .select('GWS_tavg')

# Função para calcular a média diária
def calculate_daily_mean(ano, mes, dia):
    start = ee.Date.fromYMD(ano, mes, dia)
    end = start.advance(1, 'day')

    daily_images = dataset.filterDate(start, end)
    daily_mean = daily_images.mean().clip(aoi)

    return daily_mean.set('ano', ano).set('mes', mes).set('dia', dia)

# Função para exportar imagens para o Google Drive
def export_image_to_drive(image, folder, description_prefix):
    ano = image.get('ano').getInfo()
    mes = image.get('mes').getInfo()
    dia = image.get('dia').getInfo()

    descricao = f'{description_prefix}_{ano:04d}_{mes:02d}_{dia:02d}'

    task = ee.batch.Export.image.toDrive(
        image=image,
        description=descricao,
        folder='GWS_UY',
        scale=27830,  # Ajuste a escala conforme necessário
        region=region,
        maxPixels=1e13
    )
    task.start()
    print(f'Exportação iniciada para: {descricao}')

# Itera sobre anos, meses e dias, calcula a média diária e exporta as imagens
for ano in anos:
    for mes in range(1, 13):
        days_in_month = ee.Date.fromYMD(ano, mes, 1).advance(1, 'month').difference(ee.Date.fromYMD(ano, mes, 1), 'day').getInfo()
        for dia in range(1, int(days_in_month) + 1):
            mean_image = calculate_daily_mean(ano, mes, dia)
            export_image_to_drive(mean_image, 'GWS_UY', 'GWS_Diario_UY')
